In [29]:
import tensorflow as tf 
from tensorflow.keras.datasets import cifar100
from tensorflow.keras import layers, optimizers, datasets, Sequential
import os

In [30]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.random.set_seed(2345)

In [31]:
model = Sequential([
    #unit1
    layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu ),
    layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu ),
    layers.MaxPooling2D( pool_size=[2, 2], strides=2, padding="same"),

    #unit2
    layers.Conv2D(128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu ),
    layers.Conv2D(128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu ),
    layers.MaxPooling2D( pool_size=[2, 2], strides=2, padding="same"),

    #unit3
    layers.Conv2D(256, kernel_size=[3, 3], padding="same", activation=tf.nn.relu ),
    layers.Conv2D(256, kernel_size=[3, 3], padding="same", activation=tf.nn.relu ),
    layers.MaxPooling2D( pool_size=[2, 2], strides=2, padding="same"),

    #unit4
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu ),
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu ),
    layers.MaxPooling2D( pool_size=[2, 2], strides=2, padding="same"),

    #unit5
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu ),
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu ),
    layers.MaxPooling2D( pool_size=[2, 2], strides=2, padding="same"),

])

In [32]:
model.build(input_shape=[None, 32, 32, 3])
#显然， 图片由[32, 32, 3]——》[1, 1, 512]
x = tf.random.normal([4, 32, 32, 3])
out = model(x)
print(out.shape)

(4, 1, 1, 512)


In [33]:
fc_net = Sequential([
    layers.Dense(256, activation = tf.nn.relu),
    layers.Dense(128, activation = tf.nn.relu),
    layers.Dense(100, activation = tf.nn.relu),
])

In [34]:
fc_net.build(input_shape=[None, 512])


In [35]:
variables = model.trainable_variables + fc_net.trainable_variables
optimizer = optimizers.Adam(lr = 1e-4)

In [36]:
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32)/255.0
    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [37]:
(x, y), (x_test, y_test) = datasets.cifar100.load_data()
print(x.shape, y.shape, x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [38]:
y = tf.squeeze(y, axis=1)
y_test = tf.squeeze(y_test, axis=1)

In [39]:
print(y.shape, y_test.shape)

(50000,) (10000,)


In [40]:
train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.map(preprocess).shuffle(1000).batch(batch_size=200)

In [41]:
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(batch_size=200)

In [42]:
for epoch in range(50):
    for step , (x, y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            # [None, 32, 32, 3] --> [None, 1, 1, 512]
            out = model(x)
            #squeeze [None, 1, 1, 512] --> [None, 512]
            out = tf.reshape(out, [-1, 512])
            # [None, 512] --> [None, 100]
            logits = fc_net(out)
            # --> [None, 100]]
            y_onehot = tf.one_hot(y, depth=100)
            #comput loss
            loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
            loss = tf.reduce_mean(loss)
        
        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))
        print(epoch, step, 'loss:', float(loss))
    
    total_num = 0
    total_correct = 0
    for x, y in test_db:
        out = model(x)
        out = tf.reshape(out, [-1, 512])
        logits = fc_net(out)
        prob = tf.nn.softmax(logits)
        pred = tf.argmax(prob, axis=1)
        pred = tf.cast(pred, dtype = tf.int32)

        correct = tf.cast(tf.equal(pred, y), dtype = tf.int32)

        correct = tf.reduce_sum(correct)
        total_num += x.shape[0]
        total_correct += int(correct)
    
    acc = total_correct / total_num
    print(epoch, 'acc:', acc) 


0 0 loss: 4.605147838592529
0 1 loss: 4.605076789855957
0 2 loss: 4.605535984039307
0 3 loss: 4.605108261108398
0 4 loss: 4.60533332824707
0 5 loss: 4.605353832244873
0 6 loss: 4.604804515838623
0 7 loss: 4.605074882507324
0 8 loss: 4.6052775382995605
0 9 loss: 4.604689598083496
0 10 loss: 4.604415416717529
0 11 loss: 4.605038642883301
0 12 loss: 4.604369163513184
0 13 loss: 4.605161190032959
0 14 loss: 4.603879928588867
0 15 loss: 4.604523658752441
0 16 loss: 4.6036152839660645
0 17 loss: 4.606637477874756
0 18 loss: 4.609000205993652
0 19 loss: 4.602332592010498
0 20 loss: 4.603205680847168
0 21 loss: 4.603903293609619
0 22 loss: 4.60536527633667
0 23 loss: 4.602700710296631
0 24 loss: 4.606797695159912
0 25 loss: 4.603208065032959
0 26 loss: 4.60472297668457
0 27 loss: 4.601294994354248
0 28 loss: 4.6092071533203125
0 29 loss: 4.605212211608887
0 30 loss: 4.604219436645508
0 31 loss: 4.608309268951416
0 32 loss: 4.608896255493164
0 33 loss: 4.599882125854492
0 34 loss: 4.60202264785